In [1]:
#导入包

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data.dataloader import DataLoader


In [2]:
#构建 transforms

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

In [3]:
#下载 加载数据集

trainset = datasets.CIFAR10(root='./CIFAR10', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./CIFAR10', train=False, download=True, transform=transform)

BATCH_SIZE = 128
device = "cuda"

train_loader = DataLoader(trainset,batch_size = BATCH_SIZE, shuffle = True, num_workers = 16, pin_memory = True)
test_loader = DataLoader(testset,batch_size = BATCH_SIZE, shuffle = True, num_workers = 16, pin_memory = True)

Files already downloaded and verified
Files already downloaded and verified


/root/miniconda3/envs/myconda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 3, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [4]:
#定义网络模型

class RNN_Net(nn.Module):
    def __init__(self):
        super(RNN_Net,self).__init__()
        self.hidden_dim = 128
        self.layer_dim = 3
        self.rnn = nn.RNN(32*3, 128, 3, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(128, 10)
        
    def forward(self, x):
        # （layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_().to(device)
        # 分离隐藏状态，避免梯度爆炸
        out, hn = self.rnn(x, h0.detach().cuda())
        out = self.fc(out[:, -1, :])
        return out
    
# 创建模型

net = RNN_Net().to(device)

In [5]:
# 定义优化器和损失函数

criterion = nn.CrossEntropyLoss() # 交叉式损失函数
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # 优化器

In [ ]:
#训练

EPOCHS = 200

for epoch in range(EPOCHS):
    
    train_loss = 0.0
    
    for i, (datas, labels) in enumerate(train_loader):
        # 一个batch的数据转换为RNN的输入维度
        #32*32*3
        #数据处理
        datas = datas.view(-1, 32, 32*3).requires_grad_().to(device) 
        labels = labels.to(device)
        # 梯度置零
        optimizer.zero_grad()
        # 训练
        outputs = net(datas)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        # 参数更新
        optimizer.step()
        
        train_loss += loss.item()
    print("Epoch :%d , Loss : %.3f"%(epoch+1, train_loss/len(train_loader.dataset)))

Epoch :1 , Loss : 0.018
Epoch :2 , Loss : 0.018
Epoch :3 , Loss : 0.018
Epoch :4 , Loss : 0.017
Epoch :5 , Loss : 0.017
Epoch :6 , Loss : 0.016
Epoch :7 , Loss : 0.015
Epoch :8 , Loss : 0.015
Epoch :9 , Loss : 0.014
Epoch :10 , Loss : 0.014
Epoch :11 , Loss : 0.014
Epoch :12 , Loss : 0.013
Epoch :13 , Loss : 0.013
Epoch :14 , Loss : 0.013
Epoch :15 , Loss : 0.013
Epoch :16 , Loss : 0.012
Epoch :17 , Loss : 0.012
Epoch :18 , Loss : 0.012
Epoch :19 , Loss : 0.012
Epoch :20 , Loss : 0.012
Epoch :21 , Loss : 0.012
Epoch :22 , Loss : 0.012
Epoch :23 , Loss : 0.011
Epoch :24 , Loss : 0.011
Epoch :25 , Loss : 0.011
Epoch :26 , Loss : 0.011
Epoch :27 , Loss : 0.011
Epoch :28 , Loss : 0.011
Epoch :29 , Loss : 0.011
Epoch :30 , Loss : 0.011
Epoch :31 , Loss : 0.010
Epoch :32 , Loss : 0.010
Epoch :33 , Loss : 0.010
Epoch :34 , Loss : 0.010
Epoch :43 , Loss : 0.010
Epoch :44 , Loss : 0.009
Epoch :45 , Loss : 0.009
Epoch :46 , Loss : 0.009
Epoch :47 , Loss : 0.009
Epoch :48 , Loss : 0.009
Epoch :49

In [ ]:
# 测试
correct = 0
total = 0
with torch.no_grad():
    for i , (datas, labels) in enumerate(test_loader):
        datas = datas.view(-1, sequence_dim, input_dim).to(device)
        outputs = net(datas)
        _, predicted = torch.max(outputs.data, dim=1) # 第一个是值的张量，第二个是序号的张量
        total += labels.size(0)
        correct += (predicted.cuda() == labels.cuda()).sum()
    print("Total Accuracy：{:.3f}%".format(correct / total * 100))

In [ ]:
# 显示每一类预测的概率
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

class_correct = list(0. for i in range(10))
total = list(0. for i in range(10))

with torch.no_grad():
    for (images, labels) in test_loader:
        images = images.view(-1, 32, 32*3).to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, dim=1) # 获取到每一行最大值的索引
        c = (predicted.cuda() == labels.cuda()).squeeze() # squeeze() 去掉0维【默认】， unsqueeze() 增加一维
        if labels.shape[0] == 128:
            for i in range(BATCH_SIZE):
                label = labels[i] # 获取每一个label
                class_correct[label] += c[i].item() # 累计为True的个数, 注意：1 + True = 2, 1 + False = 1
                total[label] += 1 # 该类总的个数
            
# 输出正确率
for i in range(10):
    print("Accuracy ： %5s : %2d %%" % (classes[i], 100 * class_correct[i] / total[i]))